# Analise_Filmes

Aluno: mig
Descrição: Versão refatorada para estudo — nomes padronizados e comentários ajustados.



In [8]:
import pandas as pd
import numpy as np

In [5]:
filmes = pd.read_csv('https://huggingface.co/spaces/Kamand/Movie_Recommendation/resolve/main/movies_metadata.csv',
                     low_memory = False)
filmes.head(3)

,adult,belongs_to_collection,budget,genres,homepage,id,imdb_id,original_language,original_title,overview,...,release_date,revenue,runtime,spoken_languages,status,tagline,title,video,vote_average,vote_count
0,False,"{'id': 10194, 'name': 'Toy Story Collection', ...",30000000,"[{'id': 16, 'name': 'Animation'}, {'id': 35, '...",http://toystory.disney.com/toy-story,862,tt0114709,en,Toy Story,"Led by Woody, Andy's toys live happily in his ...",...,1995-10-30,373554033.0,81.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,NaN,Toy Story,False,7.7,5415.0
1,False,NaN,65000000,"[{'id': 12, 'name': 'Adventure'}, {'id': 14, '...",NaN,8844,tt0113497,en,Jumanji,When siblings Judy and Peter discover an encha...,...,1995-12-15,262797249.0,104.0,"[{'iso_639_1': 'en', 'name': 'English'}, {'iso...",Released,Roll the dice and unleash the excitement!,Jumanji,False,6.9,2413.0
2,False,"{'id': 119050, 'name': 'Grumpy Old Men Collect...",0,"[{'id': 10749, 'name': 'Romance'}, {'id': 35, ...",NaN,15602,tt0113228,en,Grumpier Old Men,A family wedding reignites the ancient feud be...,...,1995-12-22,0.0,101.0,"[{'iso_639_1': 'en', 'name': 'English'}]",Released,Still Yelling. Still Fighting. Still Ready for...,Grumpier Old Men,False,6.5,92.0


In [6]:
avaliacoes = pd.read_csv('https://huggingface.co/spaces/Kamand/Movie_Recommendation/resolve/main/ratings_small.csv')
avaliacoes.head()

,userId,movieId,rating,timestamp
0,1,31,2.5,1260759144
1,1,1029,3.0,1260759179
2,1,1061,3.0,1260759182
3,1,1129,2.0,1260759185
4,1,1172,4.0,1260759205


In [7]:
# Filtrando somente as colunas necessários e renomeando nome das variaveis

# Seleciona somente as variaveis que iremos utilizar
filmes = filmes [['id','original_title','original_language','vote_count']]

# Renomeia as variaveis
filmes.rename(columns = {'id':'ID_FILME','original_title':'TITULO','original_language':'LINGUAGEM','vote_count':'QT_AVALIACOES'}, inplace = True)

# Exibe as primeiras linhas do arquivo tratado
filmes.head()

,ID_FILME,TITULO,LINGUAGEM,QT_AVALIACOES
0,862,Toy Story,en,5415.0
1,8844,Jumanji,en,2413.0
2,15602,Grumpier Old Men,en,92.0
3,31357,Waiting to Exhale,en,34.0
4,11862,Father of the Bride Part II,en,173.0


In [9]:
# Verificação de valores nulos
filmes.isnull().sum()

# Remoção dos nulos (poucos valores)
filmes.dropna(inplace = True)

# Verificação de valores nulos
avaliacoes.isnull().sum()

# Avaliação por usuários
avaliacoes['userId'].value_counts()

,count
userId,
547,2391
564,1868
624,1735
15,1700
73,1610
...,...
209,20
35,20
76,20


In [11]:
# Pegando usuários com mais de 999 avaliações
qt_avaliacoes = avaliacoes['userId'].value_counts() > 999
y = qt_avaliacoes[qt_avaliacoes].index
y.shape

# Visualização
y

# Tamanho do dataset
avaliacoes.shape

avaliacoes = avaliacoes[avaliacoes['userId'].isin(y)]

avaliacoes.shape

(15028, 4)

In [12]:
y

Index([547, 564, 624, 15, 73, 452, 468, 380, 311, 30], dtype='int64', name='userId')

In [14]:
# Visualizando Df filmes
filmes.head()

# Pegando filmes com mais de 999 avaliações
filmes = filmes[filmes['QT_AVALIACOES'] > 999]

# Agrupamento de filmes por linguagem
filmes_linguagem = filmes['LINGUAGEM'].value_counts()
filmes_linguagem.head(20)
filmes = filmes[filmes['LINGUAGEM'] == 'en']

# Visualizando os tipos dos dados
filmes.info()
avaliacoes.info()

<class 'pandas.core.frame.DataFrame'>
Index: 1100 entries, 0 to 44842
Data columns (total 4 columns):
 #   Column         Non-Null Count  Dtype  
---  ------         --------------  -----  
 0   ID_FILME       1100 non-null   object 
 1   TITULO         1100 non-null   object 
 2   LINGUAGEM      1100 non-null   object 
 3   QT_AVALIACOES  1100 non-null   float64
dtypes: float64(1), object(3)
memory usage: 43.0+ KB
<class 'pandas.core.frame.DataFrame'>
Index: 15028 entries, 962 to 95268
Data columns (total 4 columns):
 #   Column     Non-Null Count  Dtype  
---  ------     --------------  -----  
 0   userId     15028 non-null  int64  
 1   movieId    15028 non-null  int64  
 2   rating     15028 non-null  float64
 3   timestamp  15028 non-null  int64  
dtypes: float64(1), int64(3)
memory usage: 587.0 KB


In [16]:
# Convertendo o ID_FILME em int
filmes['ID_FILME'] = filmes['ID_FILME'].astype(int)

# Concatenando os df
avaliacoes_e_filmes = avaliacoes.merge(filmes, left_on= 'movieId', right_on= 'ID_FILME')
avaliacoes_e_filmes.head()

,userId,movieId,rating,timestamp,ID_FILME,TITULO,LINGUAGEM,QT_AVALIACOES
0,15,11,2.5,1093028381,11,Star Wars,en,6778.0
1,15,14,2.5,1166586286,14,American Beauty,en,3438.0
2,15,22,2.5,1093070415,22,Pirates of the Caribbean: The Curse of the Bla...,en,7191.0
3,15,62,2.0,1093028336,62,2001: A Space Odyssey,en,3075.0
4,15,70,0.5,1093070467,70,Million Dollar Baby,en,2519.0


In [17]:
# Verificando tamanho do arq
avaliacoes_e_filmes.shape

# Verif. valores nulos
avaliacoes_e_filmes.isna().sum()

avaliacoes_e_filmes.head(20)

,userId,movieId,rating,timestamp,ID_FILME,TITULO,LINGUAGEM,QT_AVALIACOES
0,15,11,2.5,1093028381,11,Star Wars,en,6778.0
1,15,14,2.5,1166586286,14,American Beauty,en,3438.0
2,15,22,2.5,1093070415,22,Pirates of the Caribbean: The Curse of the Bla...,en,7191.0
3,15,62,2.0,1093028336,62,2001: A Space Odyssey,en,3075.0
4,15,70,0.5,1093070467,70,Million Dollar Baby,en,2519.0
5,15,95,1.5,1093028331,95,Armageddon,en,2540.0
6,15,107,2.0,1166586594,107,Snatch,en,2953.0
7,15,111,5.0,997938500,111,Scarface,en,3017.0
8,15,153,1.0,1128274517,153,Lost in Translation,en,1943.0
9,15,161,3.0,1093028319,161,Ocean's Eleven,en,3857.0


In [19]:
# Pivot. avaliações por usuario
filmes_pivot = avaliacoes_e_filmes.pivot_table(index = 'userId', columns = 'TITULO', values = 'rating')
filmes_pivot.head(20)

# Preencher valores nulos com 0
filmes_pivot.fillna(0, inplace = True)
filmes_pivot.head()

TITULO,10 Things I Hate About You,1408,2001: A Space Odyssey,28 Days Later,28 Weeks Later,300,4: Rise of the Silver Surfer,8 Mile,A Beautiful Mind,A Clockwork Orange,...,Underworld,Valkyrie,Van Helsing,Vertigo,War of the Worlds,What Women Want,Wild Wild West,X-Men Origins: Wolverine,Zodiac,xXx
userId,,,,,,,,,,,,,,,,,,,,,
15,0.0,0.0,2.0,2.5,1.5,0.0,0.0,0.0,0.0,2.0,...,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,3.5
30,3.0,0.0,0.0,4.0,3.0,4.0,0.0,0.0,0.0,0.0,...,0.0,3.0,0.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0
73,0.0,0.0,0.0,0.0,0.5,0.0,0.0,2.0,0.0,2.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5,0.0,0.0
311,0.0,0.0,2.5,0.0,0.0,3.0,0.0,0.0,2.0,2.5,...,3.0,0.0,0.0,0.0,3.0,0.0,3.0,3.5,4.0,0.0
380,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.5,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3.5


In [20]:
# Método de criação da matriz sparsa
from scipy.sparse import csr_matrix

filmes_sparse = csr_matrix(filmes_pivot)

type(filmes_sparse)

from sklearn.neighbors import NearestNeighbors

# Criando e treinando
modelo = NearestNeighbors(algorithm = 'brute')
modelo.fit(filmes_sparse)

NearestNeighbors(algorithm='brute')

In [21]:
print("Aqui estão alguns filmes que foram avaliados:")
print(filmes_pivot.index[:20].to_list())

Here are some movie titles from the list:
[15, 30, 73, 311, 380, 452, 468, 547, 564, 624]


In [24]:
if '1408' in filmes_pivot.index:
    distances, suggestions = modelo.kneighbors(filmes_pivot.filter(items= ['1408'], axis=0).values.reshape(1,-1))

    for i in range(len(suggestions)):
        print(filmes_pivot.index[suggestions[i]])
else:
    print("ESte filme não está na lista de filmes.")

This film not found in the movie list.
